In [1]:
import pandas as pd
import numpy as np
import os
import datetime
os.getcwd()

# Note email cpm updated based on the Q4 client number
# Flipp hosted not included in wide version

'/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3'

In [2]:
TMR_up_to_2019Q2=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/BL_MMM_long_cumu_up_to_2019Q2_JL_2019-11-15.csv",dtype=str)
TMR_up_to_2019Q2['impression']=TMR_up_to_2019Q2['impression'].astype(float).astype(int)
TMR_up_to_2019Q2['click']=TMR_up_to_2019Q2['click'].astype(float).astype(int)
TMR_up_to_2019Q2['cost']=TMR_up_to_2019Q2['cost'].astype(float)


In [3]:
TMR_up_to_2019Q2_no_DMA=TMR_up_to_2019Q2.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2019Q2_no_DMA['week date'].min())
print(TMR_up_to_2019Q2_no_DMA['week date'].max())
# TMR_up_to_2019Q2_no_DMA

2016-09-25
2019-07-28


In [4]:
TMR_up_to_2019Q2_no_DMA.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2016-09-25,Email,xx,xx,11581550,113834,5278.691191
1,2016-10-02,Digital,Flipp,Flipp App,74691,37222,10794.380000


In [5]:
TMR_up_to_2019Q2_no_DMA.shape

(6524, 7)

In [6]:
def get_week_end_date_BL(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

In [7]:
tv_18Q3_qc=TMR_up_to_2019Q2_no_DMA[TMR_up_to_2019Q2_no_DMA['media']=="TV"]
tv_18Q3_qc=tv_18Q3_qc[tv_18Q3_qc['week date']>="2018-08-04"]
tv_18Q3_qc=tv_18Q3_qc[tv_18Q3_qc['week date']<="2018-11-03"]

print(tv_18Q3_qc['week date'].nunique())
print(tv_18Q3_qc['week date'].unique())
print(tv_18Q3_qc['impression'].sum())
print(tv_18Q3_qc['cost'].sum())


5
['2018-08-12' '2018-08-19' '2018-08-26' '2018-09-02' '2018-10-28']
85485096
1515200.3429999999


# TV

In [8]:
# Email: Big Lots 2019Q3 TMR data about TV and Circulation from Faith, Friday, February 21, 2020 at 11:02 AM
'''
Media                                                   Total Net
National Cable (DR)                         $340,349 (includes spots in DRTV Market and local Continuum)
National Cable                              $650,096

# The unrated cleo is under DRTV, but since it's unrated and confirmed with Faith on 0228, it's OK to ignore this part
'''
TV_TMR_data=pd.DataFrame()

In [9]:
TV_logs=pd.ExcelFile("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/input_2019Q3/Big Lots Q3 2019 National TMR - 3.3.2020.xlsx")
print(TV_logs.sheet_names)
df_TV_national=TV_logs.parse("Nat Cable 2019",dtype=str,skiprows=2)
df_TV_local=TV_logs.parse("Local",dtype=str,skiprows=2)

print(df_TV_national.shape,df_TV_local.shape)

['Nat Cable 2019', 'Local', 'MS', 'DRTV 2019', 'Cleo TV', 'Details 8.26', 'Details 9.2']
(1125, 19) (71895, 11)


In [10]:
df_TV_national=df_TV_national[["Q3 2019",'National',"Network","Air Date",' Imp - Women 25-54']]
df_TV_national=df_TV_national.rename(columns={"Network":"placement"})
df_TV_national['date']=df_TV_national['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
df_TV_national['impression']=df_TV_national[' Imp - Women 25-54'].astype(float)

df_TV_national['media']="TV"
df_TV_national['submedia']="National"
df_TV_national['click']=0
df_TV_national['week date']=df_TV_national['date'].apply(get_week_end_date_BL)
df_TV_national['week date']=df_TV_national['week date'].apply(lambda x: x-datetime.timedelta(days=6))

df_TV_national_Cable=df_TV_national[df_TV_national['National']=="National"]
sum_national_cable_impr=df_TV_national_Cable['impression'].sum()
print("sum_national_cable_impr",sum_national_cable_impr)
# 650,096 from email, Faith, Friday, February 28, 2020 at 5:41 PM
df_TV_national_Cable['cost']=df_TV_national_Cable['impression']/sum_national_cable_impr*650096
df_TV_national_Cable=df_TV_national_Cable[TMR_up_to_2019Q2_no_DMA.columns.tolist()]
print("df_TV_national_Cable['cost'].sum()",df_TV_national_Cable['cost'].sum())


sum_national_cable_impr 42761695.0
df_TV_national_Cable['cost'].sum() 650096.0


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [11]:
df_TV_national_DRTV=df_TV_national[df_TV_national['National']!="National"]
print(df_TV_national_DRTV['National'].unique())

df_TV_local=df_TV_local[['Network','Air Date',' Imp - Women 25-54']].rename(columns={" Imp - Women 25-54":"impression"})
df_TV_local['impression']=df_TV_local['impression'].astype(float)
df_TV_local['date']=df_TV_local['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())


df_TV_local=df_TV_local.rename(columns={"Network":"placement"})
df_TV_local['media']="TV"
df_TV_local['submedia']="Local"
df_TV_local['click']=0
df_TV_local['week date']=df_TV_local['date'].apply(get_week_end_date_BL)
df_TV_local['week date']=df_TV_local['week date'].apply(lambda x: x-datetime.timedelta(days=6))


df_TV_local_and_DTV=df_TV_local.append(df_TV_national_DRTV)
sum_local_and_DTV_impr=df_TV_local_and_DTV['impression'].sum()
print("sum_local_and_DTV_impr",sum_local_and_DTV_impr)
# 340,349 from email, Faith, Friday, February 28, 2020 at 5:41 PM
df_TV_local_and_DTV['cost']=df_TV_local_and_DTV['impression']/sum_local_and_DTV_impr*340349

df_TV_local_and_DTV=df_TV_local_and_DTV[TMR_up_to_2019Q2_no_DMA.columns.tolist()]
print("df_TV_local_and_DTV['cost'].sum()",df_TV_local_and_DTV['cost'].sum())


['DRTV']
sum_local_and_DTV_impr 21392365.312499244
df_TV_local_and_DTV['cost'].sum() 340349.0


/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [12]:
TV_TMR_data=df_TV_national_Cable.append(df_TV_local_and_DTV)
TV_TMR_data['week date']=TV_TMR_data['week date'].astype(str)
print("TV_TMR_data['cost'].sum()",TV_TMR_data['cost'].sum())
print("TV_TMR_data['impression'].sum()",TV_TMR_data['impression'].sum())

df_TV_summary=TV_TMR_data.groupby(['submedia'])['impression','cost'].sum().reset_index()
df_TV_summary['impression']=df_TV_summary['impression'].astype(int)


TV_TMR_data['cost'].sum() 990445.0
TV_TMR_data['impression'].sum() 64154060.31249925


# Others

In [13]:
Others_TMR_week=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/input_2019Q3/DatoramaData_2019Q3_0804-1102.xlsx",
                              dtype=str,sheet_name="TMR")
Others_TMR_week=Others_TMR_week.rename(columns={"Week":"week date","Media":"media","Submedia":"submedia","Placement":"placement",
                                               "Impressions":"impression","Clicks":"click","Cost":"cost"})
Others_TMR_week['week date']=Others_TMR_week['week date'].apply(lambda x: x[:10])
Others_TMR_week['impression']=Others_TMR_week['impression'].fillna(0).astype(float).astype(int)
Others_TMR_week['click']=Others_TMR_week['click'].fillna(0).astype(float).astype(int)
Others_TMR_week['cost']=Others_TMR_week['cost'].fillna(0).astype(float)


### Revise Emails

In [14]:
Others_TMR_week_1=Others_TMR_week[Others_TMR_week['media']!="Email"]
Others_TMR_week_2=Others_TMR_week[Others_TMR_week['media']=="Email"]
Others_TMR_week_2['impression'].sum(),Others_TMR_week_2['cost'].sum()

(2453671772, 1118345.60723082)

In [15]:
# Based on the 2019Q2 number
# BL - Email cost request for Media Mix model
# Email from Ollhoff, Dominic <dollhoff@biglots.com>, Friday, November 15, 2019 at 1:42 PM

# If not I show that Q2 cost was $530,690 for 2.042B e-mails.


email_cost_client=530690
email_cost_impression=2033659691 # the actual number we have
new_email_c_p_impr=email_cost_client/email_cost_impression
print(new_email_c_p_impr)
Others_TMR_week_2['cost']=Others_TMR_week_2['impression']*new_email_c_p_impr
print(new_email_c_p_impr*1000)

0.00026095319799501304
0.26095319799501304


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [16]:
print(Others_TMR_week_2['week date'].min())
print(Others_TMR_week_2['week date'].max())

Others_TMR_week=Others_TMR_week_1.append(Others_TMR_week_2)

2019-08-04
2019-10-27


## Add Print Circ

In [17]:
updated_circ_2019Q3_Faith=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/archived/Big Lots Q3 2019 National TMR - 1.13.2020.xlsx",
                                        dtype=str,skiprows=2,sheet_name="Q3 Print Circ 2019")
updated_circ_2019Q3_Faith=updated_circ_2019Q3_Faith.iloc[:,1:]
updated_circ_2019Q3_Faith=updated_circ_2019Q3_Faith.rename(columns={"Week_Start_Sunday":"week date","Circulation actual delivers":"impression","Circulation total cost":"cost"})
updated_circ_2019Q3_Faith['impression']=updated_circ_2019Q3_Faith['impression'].replace("-","0")
updated_circ_2019Q3_Faith['impression']=updated_circ_2019Q3_Faith['impression'].astype(int)
updated_circ_2019Q3_Faith['cost']=updated_circ_2019Q3_Faith['cost'].replace("-","0")
updated_circ_2019Q3_Faith['cost']=updated_circ_2019Q3_Faith['cost'].astype(float)

updated_circ_2019Q3_Faith

,week date,impression,cost
0,2019-08-04 00:00:00,0,0.000000e+00
1,2019-08-11 00:00:00,0,0.000000e+00
2,2019-08-18 00:00:00,0,0.000000e+00
3,2019-08-25 00:00:00,12478759,7.411242e+05
4,2019-09-01 00:00:00,14742974,8.539707e+05
5,2019-09-08 00:00:00,0,0.000000e+00
6,2019-09-15 00:00:00,14321485,6.923081e+05
7,2019-09-22 00:00:00,0,0.000000e+00
8,2019-09-29 00:00:00,0,0.000000e+00
9,2019-10-06 00:00:00,14569743,1.010069e+06


In [18]:
updated_circ_2019Q3_Faith['week date']=updated_circ_2019Q3_Faith['week date'].apply(lambda x: x.split(" ")[0])
updated_circ_2019Q3_Faith['click']=0
updated_circ_2019Q3_Faith['media']="Circulation"
updated_circ_2019Q3_Faith['submedia']="xx"
updated_circ_2019Q3_Faith['placement']="xx"
updated_circ_2019Q3_Faith=updated_circ_2019Q3_Faith[Others_TMR_week.columns.tolist()]

In [19]:
for col in Others_TMR_week.columns.tolist():
    print(col,Others_TMR_week[col].apply(type).unique())
    
Others_TMR_week=Others_TMR_week[Others_TMR_week['media']!="Circulation"] 
Others_TMR_week=Others_TMR_week.append(updated_circ_2019Q3_Faith)

week date [<class 'str'>]
media [<class 'str'>]
submedia [<class 'str'>]
placement [<class 'str'>]
impression [<class 'int'>]
click [<class 'int'>]
cost [<class 'float'>]


# Seperate PMP from the programmatic

In [20]:
print(Others_TMR_week['impression'].sum())
print(Others_TMR_week['click'].sum())
print(Others_TMR_week['cost'].sum())
'''
3130461058.0
28705022.0
10450581.11631801
'''

3130461058
28705022
10450581.11631801


'\n3130461058.0\n28705022.0\n10450581.11631801\n'

In [21]:
print(Others_TMR_week[Others_TMR_week['submedia']=="Programmatic"]['impression'].sum())
print(Others_TMR_week[Others_TMR_week['submedia']=="Programmatic"]['click'].sum())
print(Others_TMR_week[Others_TMR_week['submedia']=="Programmatic"]['cost'].sum())

'''
121877185
431712
952972.31940652
'''

121877185
431712
952972.31940652


'\n121877185\n431712\n952972.31940652\n'

In [22]:
# Try only this quarter
excel_file_joann=pd.ExcelFile("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/input_2019Q3/DatoramaData_2019Q3_0804-1102.xlsx")


df_digital_joann=excel_file_joann.parse("Digital",dtype=str,skiprows=4).fillna(0)
df_digital_joann['Impressions']=df_digital_joann['Impressions'].astype(float)
df_digital_joann['Clicks']=df_digital_joann['Clicks'].astype(float)
df_digital_joann['Media Cost']=df_digital_joann['Media Cost'].astype(float)
df_digital_joann=df_digital_joann[['Week','Media','Submedia','Partner','Tactic','Impressions','Clicks','Media Cost']]
df_digital_joann=df_digital_joann[df_digital_joann['Week']!="Totals"]
df_digital_joann['Week']=df_digital_joann['Week'].apply(lambda x: x.split(" ")[0])
df_digital_joann=df_digital_joann.rename(columns={"Week":"week date","Media":"media","Submedia":"submedia",
                                                  "Impressions":"impression","Clicks":"click","Media Cost":"cost"})

print("df_digital_joann['impression'].sum()",df_digital_joann['impression'].sum())
list_PMP=['Cluep','Discovery Channel','Evite','TBS.com','Waze']
df_digital_joann_1_PMP=df_digital_joann[df_digital_joann['Partner'].isin(list_PMP)]
df_digital_joann_2_Programmatic=df_digital_joann[~df_digital_joann['Partner'].isin(list_PMP)]

df_digital_joann_2_Programmatic_YouTube=df_digital_joann_2_Programmatic[df_digital_joann_2_Programmatic['Partner']=="YouTube"]
df_digital_joann_3_Programmatic_Prog=df_digital_joann_2_Programmatic[df_digital_joann_2_Programmatic['Partner']!="YouTube"]

print("df_digital_joann_1_PMP['impression'].sum()",df_digital_joann_1_PMP['impression'].sum())
print("df_digital_joann_2_Programmatic_YouTube['impression'].sum()",df_digital_joann_2_Programmatic_YouTube['impression'].sum())
print("df_digital_joann_3_Programmatic_Prog['impression'].sum()",df_digital_joann_3_Programmatic_Prog['impression'].sum())


# ! YouTube already in Joann's formatted table
df_digital_joann_1_PMP.head(2)

df_digital_joann['impression'].sum() 168429350.0
df_digital_joann_1_PMP['impression'].sum() 33933202.0
df_digital_joann_2_Programmatic_YouTube['impression'].sum() 46552165.0
df_digital_joann_3_Programmatic_Prog['impression'].sum() 87943983.0


,week date,media,submedia,Partner,Tactic,impression,click,cost
3,2019-10-27,Digital,Custom Execution,Evite,Custom Executions,0.0,0.0,0.0
4,2019-10-27,Digital,Custom Execution,Waze,Custom Executions,0.0,0.0,0.0


In [23]:
df_digital_joann_1_PMP=df_digital_joann_1_PMP.rename(columns={"Partner":"placement"})
df_digital_joann_1_PMP['submedia']="Programmatic--PMP"
df_digital_joann_1_PMP=df_digital_joann_1_PMP[Others_TMR_week.columns.tolist()]

df_digital_joann_3_Programmatic_Prog=df_digital_joann_3_Programmatic_Prog.rename(columns={"Partner":"placement"})
df_digital_joann_3_Programmatic_Prog['submedia']="Programmatic"
df_digital_joann_3_Programmatic_Prog=df_digital_joann_3_Programmatic_Prog[Others_TMR_week.columns.tolist()]

Others_TMR_week=Others_TMR_week[Others_TMR_week['submedia']!="Programmatic"]
Others_TMR_week=Others_TMR_week.append(df_digital_joann_1_PMP)
Others_TMR_week=Others_TMR_week.append(df_digital_joann_3_Programmatic_Prog)


print(Others_TMR_week['impression'].sum())
print(Others_TMR_week['click'].sum())
print(Others_TMR_week['cost'].sum())

3130461058.0
28705022.0
10450581.11631801


# Cumulative Q3 TMR

In [24]:
Q3_2019_only_TMR=Others_TMR_week.append(TV_TMR_data)
Cum_Q3_2019_TMR=TMR_up_to_2019Q2.append(Q3_2019_only_TMR)

In [25]:
try:
    os.stat("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/output/")
except:
    os.mkdir("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/output/")

Q3_2019_only_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/output/BL_MMM_2019Q3_Only_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
Cum_Q3_2019_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/output/BL_MMM_long_cumu_up_to_2019Q3_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


# Wide

In [26]:
# seperate the hosted flipp
Cum_Q3_2019_TMR_FlippHost=Cum_Q3_2019_TMR[(Cum_Q3_2019_TMR['submedia']=="Flipp") & (Cum_Q3_2019_TMR['placement']=="Hosted")]
Cum_Q3_2019_TMR_FlippHost['submedia']="Flipp(hosted)"

Cum_Q3_2019_TMR_FlippNonHost=Cum_Q3_2019_TMR[(Cum_Q3_2019_TMR['submedia']=="Flipp") & (Cum_Q3_2019_TMR['placement']!="Hosted")]
Cum_Q3_2019_TMR_FlippNonHost['submedia']="Flipp(nonhosted)"


Cum_Q3_2019_TMR=Cum_Q3_2019_TMR[(Cum_Q3_2019_TMR['submedia']!="Flipp")]
Cum_Q3_2019_TMR['submedia']=np.where(Cum_Q3_2019_TMR['media']=="Circulation","xx",Cum_Q3_2019_TMR['submedia'])


Cum_Q3_2019_TMR=Cum_Q3_2019_TMR.append(Cum_Q3_2019_TMR_FlippHost).append(Cum_Q3_2019_TMR_FlippNonHost)


########################

data_media_national=Cum_Q3_2019_TMR.groupby(['week date','media'])['impression','click','cost'].sum().reset_index()
data_submedia_national=Cum_Q3_2019_TMR.groupby(['week date','media','submedia'])['impression','click','cost'].sum().reset_index()
data_submedia_national['media_submedia']=data_submedia_national['media']+"-"+data_submedia_national['submedia']
data_submedia_national=data_submedia_national[['week date','media_submedia','impression','click','cost']]


data_media_national_wide=data_media_national[['week date']].drop_duplicates()
for col in data_media_national.columns.tolist()[-3:]:
    df=data_media_national[['week date','media']+[col]].pivot(index='week date',columns='media',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_media_national_wide=pd.merge(data_media_national_wide,df,on='week date',how="outer")
data_media_national_wide=data_media_national_wide.fillna(0)
data_media_national_wide=data_media_national_wide.drop_duplicates()
#############

data_submedia_national_wide=data_submedia_national[['week date']].drop_duplicates()
for col in data_submedia_national.columns.tolist()[-3:]:
    df=data_submedia_national[['week date','media_submedia']+[col]].pivot(index='week date',columns='media_submedia',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_submedia_national_wide=pd.merge(data_submedia_national_wide,df,on='week date',how="outer")
data_submedia_national_wide=data_submedia_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.drop_duplicates()


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
Cum_Q3_2019_TMR['click'].apply(lambda x: type(x)).unique()

array([<class 'float'>], dtype=object)

In [28]:
including_weeks=[str(datetime.date(2019,11,2)-datetime.timedelta(days=7*x+6)) for x in range(104)]


data_media_national_wide=data_media_national_wide.sort_values("week date")
data_media_national_wide=data_media_national_wide[data_media_national_wide['week date'].isin(including_weeks)]

data_submedia_national_wide=data_submedia_national_wide.sort_values("week date")
data_submedia_national_wide=data_submedia_national_wide[data_submedia_national_wide['week date'].isin(including_weeks)]



## Sales

In [1]:
sales_data=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2020-02-29.csv",dtype=str)
sales_data=sales_data[sales_data['week_end_date']<="2019-11-02"]
sales_data['week_end_date']=sales_data['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_data['sales']=sales_data['sales'].astype(float)
sales_data['transactions']=sales_data['transactions'].astype(float)
sales_data['week date']=sales_data['week_end_date'].apply(lambda x: x-datetime.timedelta(days=6))
sales_data_naitonal=sales_data.groupby(['week date'])['sales','transactions'].sum().reset_index()
sales_data_naitonal=sales_data_naitonal.set_index(['week date'])
sales_data_naitonal.at[datetime.date(2017,4,30),'transactions']=(sales_data_naitonal.at[datetime.date(2017,4,23),'transactions']+sales_data_naitonal.at[datetime.date(2017,5,7),'transactions'])/2
sales_data_naitonal=sales_data_naitonal.reset_index()
sales_data_store_counts=sales_data[sales_data['sales']>0]
sales_data_store_counts=sales_data_store_counts.groupby(['week date'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_counts"})

sales_data_naitonal=pd.merge(sales_data_naitonal,sales_data_store_counts,on="week date",how="outer")
sales_data_naitonal['week date']=sales_data_naitonal['week date'].astype(str)

NameError: name 'pd' is not defined

In [30]:
data_media_national_wide=pd.merge(data_media_national_wide,sales_data_naitonal,on="week date",how="left")
data_submedia_national_wide=pd.merge(data_submedia_national_wide,sales_data_naitonal,on="week date",how="left")

## Promotion

In [31]:
Rewards_Promotion_list=[datetime.date(2016,10,1),datetime.date(2016,10,2),
                        datetime.date(2017,1,21),datetime.date(2017,1,22),
                        datetime.date(2017,4,1),datetime.date(2017,4,2),
                        datetime.date(2017,7,8),datetime.date(2017,7,9),
                        datetime.date(2017,9,30),datetime.date(2017,10,1),
                        datetime.date(2018,1,20),datetime.date(2018,1,21),
                        datetime.date(2018,4,7),datetime.date(2018,4,8),
                        datetime.date(2018,7,7),datetime.date(2018,7,8),
                        datetime.date(2018,9,29),datetime.date(2018,9,30),
                        datetime.date(2019,1,26),datetime.date(2019,1,27),  
                        datetime.date(2019,4,6),datetime.date(2019,4,7),
                        datetime.date(2019,7,13),datetime.date(2019,7,14),
                        datetime.date(2019,9,28),datetime.date(2019,9,29),
                       ]

df_Rewards_Promotion=pd.DataFrame({"Date":Rewards_Promotion_list},index=range(len(Rewards_Promotion_list)))
df_Rewards_Promotion['weekday']=df_Rewards_Promotion['Date'].apply(lambda x: x.weekday())
df_Rewards_Promotion['week date']=np.where(df_Rewards_Promotion['weekday']==6,df_Rewards_Promotion['Date'],df_Rewards_Promotion['Date']-datetime.timedelta(days=6))

del df_Rewards_Promotion['Date']

df_Rewards_Promotion_Sunday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==6]
df_Rewards_Promotion_Sunday['Sunday_rewards_ind']=1
del df_Rewards_Promotion_Sunday['weekday']
df_Rewards_Promotion_Sunday['week date']=df_Rewards_Promotion_Sunday['week date'].astype(str)


df_Rewards_Promotion_Saturday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==5]
df_Rewards_Promotion_Saturday['Saturday_rewards_ind']=1
del df_Rewards_Promotion_Saturday['weekday']
df_Rewards_Promotion_Saturday['week date']=df_Rewards_Promotion_Saturday['week date'].astype(str)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.or

In [32]:
holiday_1_week_df=pd.DataFrame({"week date":[datetime.date(2016,12,18),datetime.date(2017,12,17),datetime.date(2018,12,23)],"Holiday_1_week_only_Ind":[1]*3},index=[0,1,2])
holiday_5_weeks_df=pd.DataFrame({"week date":[datetime.date(2016,12,18)-datetime.timedelta(days=x*7) for x in range(5)] +[datetime.date(2017,12,17)-datetime.timedelta(days=x*7) for x in range(5)]+[datetime.date(2018,12,23)-datetime.timedelta(days=x*7) for x in range(5)],
                                 "Holiday_5_weeks_only_Ind":[1]*15},index=[x for x in range(15)])

holiday_1_week_df['week date']=holiday_1_week_df['week date'].astype(str)
holiday_5_weeks_df['week date']=holiday_5_weeks_df['week date'].astype(str)

df_special_weeks_ind=pd.merge(df_Rewards_Promotion_Saturday,df_Rewards_Promotion_Sunday,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_1_week_df,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_5_weeks_df,on="week date",how="outer")


In [33]:
data_media_national_wide=pd.merge(data_media_national_wide,df_special_weeks_ind,on="week date",how="left")

data_submedia_national_wide=pd.merge(data_submedia_national_wide,df_special_weeks_ind,on="week date",how="left")

data_media_national_wide=data_media_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.fillna(0)

# Adjust the 2019Q1 end week

In [34]:
data_submedia_national_wide['Dummy_2019Q1_End_2weeks']=np.where(data_submedia_national_wide['week date'].isin(["2019-04-21","2019-04-28"]),1,0)


In [35]:
data_media_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/output/BL_MMM_data_media_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
data_submedia_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q3/output/BL_MMM_data_submedia_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

# Note email cpm updated based on the Q2 client number, for Q3, West confirmed it's close
# Flipp hosted seperated in wide version
# Circ confirmed with Dom

# Flipp the reads not same as West's number, but it's not updated since not applied in model building. 